In [ ]:
from simple_unet_model import simple_unet_model 
from keras.utils import normalize
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from skimage import measure, color, io


IMG_HEIGHT = 256
IMG_WIDTH  = 256
IMG_CHANNELS = 1

def get_model():
    return simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

# load the model and corresponding weights

model = get_model()
model.load_weights('file.hdf5')

# load and process the test image - image that needs to be segmented. 
test_img = cv2.imread('C:/Users/.tiff', 0)
test_img = Image.fromarray(test_img)
test_img = test_img.resize((IMG_HEIGHT, IMG_WIDTH))
test_img_norm = np.expand_dims(normalize(np.array(test_img), axis=1),2)
test_img_norm=test_img_norm[:,:,0][:,:,None]
test_img_input=np.expand_dims(test_img_norm, 0)

# predict and threshold for values above 0.7 probability
segmented = (model.predict(test_img_input)[0,:,:,0] > 0.7).astype(np.uint8)

# print testing and segmented image
plt.figure(figsize=(8, 8))
plt.subplot(221)
plt.title('Testing Image')
plt.imshow(test_img, cmap='gray')
plt.subplot(222)
plt.title('Segmented Image')
plt.imshow(segmented, cmap='gray')
plt.show()

# save U-NET segmented image
plt.imsave('C:/Users/.jpg', segmented, cmap='gray')

In [ ]:
# https://scikit-image.org/docs/dev/auto_examples/applications/plot_human_mitosis.html

import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as ndi

from skimage import (
    color, feature, filters, measure, morphology, segmentation, util
)
from skimage.data import human_mitosis

# import U-NET segmented images
image = cv2.imread('C:/Users/.jpg')
image = image[:,:,0]

fig, ax = plt.subplots()
ax.imshow(image, cmap='gray')
plt.show()

In [ ]:
# Otsu thresholding
thresholds = filters.threshold_multiotsu(image, classes=3)
regions = np.digitize(image, bins=thresholds)

fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
ax[0].imshow(image)
ax[0].set_title('Original')
ax[0].axis('off')
ax[1].imshow(regions)
ax[1].set_title('Multi-Otsu thresholding')
ax[1].axis('off')
plt.show()

In [ ]:
# include pixels above value 0
cells = image > thresholds[0]

In [ ]:
# watershed segmentation 
distance = ndi.distance_transform_edt(cells)

local_max_coords = feature.peak_local_max(distance, min_distance=5)
local_max_mask = np.zeros(distance.shape, dtype=bool)
local_max_mask[tuple(local_max_coords.T)] = True
markers = measure.label(local_max_mask)

segmented_cells = segmentation.watershed(-distance, markers, mask=cells)

In [ ]:
# visualize segmentation
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
ax[0].imshow(cells, cmap='gray')
ax[0].set_title('Overlapping cells')
ax[0].axis('off')
ax[1].imshow(color.label2rgb(segmented_cells, bg_label=0))
ax[1].set_title('Segmented cells')
ax[1].axis('off')
plt.show()

In [ ]:
# visualize number of lamellipodia
color_labels = color.label2rgb(segmented_cells, image, alpha=0.4, bg_label=0)

fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(color_labels)
ax.set_title('Segmentation result over raw image')
plt.show()

In [ ]:
# number of lamellipodia
print(segmented_cells.max())